In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import sys
sys.path.append('../')
from networks import *

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.CIFAR10(root='mnist_data', download=True, transform=transform)

Files already downloaded and verified


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 4060 Ti


In [5]:
def KFold_Model(model):
  
  k = 10
  kf = KFold(n_splits=k, random_state=56, shuffle=True)
  accuracies = []
  torch.manual_seed(56)

  for train_idx, test_idx in kf.split(dataset):
      train_loader = DataLoader(Subset(dataset, train_idx), batch_size=64, shuffle=True)
      test_loader = DataLoader(Subset(dataset, test_idx), batch_size=64, shuffle=False)

      criterion = nn.CrossEntropyLoss()
      optimizer = optim.SGD(model.parameters(), lr=0.01)
      model = model.to(device)
      # Training
      for epoch in range(15):
          model.train()
          for images, labels in train_loader:
              images, labels = images.to(device), labels.to(device)
              optimizer.zero_grad()
              outputs = model(images)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()

      # Evaluation
      model.eval()
      correct, total = 0, 0
      with torch.no_grad():
          for images, labels in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      accuracies.append(accuracy)
      print(f'Fold Accuracy: {accuracy}%')

  print(f'Mean Accuracy: {np.mean(accuracies)}%')
  print(f'Standard Deviation: {np.std(accuracies)}%')

In [ ]:
model = NN01()
KFold_Model(model)